In [1]:
from helper_parthi2929 import *

graph = load_map_graph(romania_location_map)
#show_map(graph, start='A',goal='B',path=['A','S','F','B'],SLDTarget='B')
show_map(graph)

In [2]:
map_40 = load_map_graph(map_40_dict)
#show_map(graph, start='A',goal='B',path=['A','S','F','B'],SLDTarget='B')
show_map(map_40)

In [3]:
map_40_dict.get(5,[]).get('connections',[])

[32, 16, 14]

Since we used netowrkx, it is also possible using its power, to access neighbors as follows..

In [4]:
list(map_40.neighbors(5))

[32, 16, 14]

Let us try our A star with slight modification (that we USE networkx object). (Though given points are not exactly lat,lon Euclidean as heuristic is not good enough, giving a costlier path)
Changes are denoted by ```#CHANGE```

In [5]:
from queue import PriorityQueue
import geopy.distance

G = map_40 #CHANGE

def calculate_GD(start, end): 

    (x0,y0) = G.node[start]['pos']    #CHANGE
    (x1,y1) = G.node[end]['pos']     #CHANGE
    return floor(geopy.distance.geodesic((y0,x0), (y1,x1)).miles) 

def calculate_totalGD(node):
    cost = 0
    if cameFrom.get(node) is None: return 0
    while True:   #when current_node is not None..        
        cost += calculate_GD(cameFrom.get(node),node)
        node = cameFrom.get(node)
        if cameFrom.get(node) is None: break
    return cost

def calculate_path(node):
    path = [node]
    while node is not None:   #when current_node is not None..        
        node = cameFrom.get(node)
        path.append(node)
    return [x for x in reversed(path[:-1])]

start = 5
goal = 34
cameFrom = {}           
cameFrom[start] = None 

def run_AStar(start, goal):

    # Initialization
    q = PriorityQueue()
    q.put((0,start))
    closedSet = set()  # set() would be better than a list?
    count = 0

    
    # Main loop
    while not q.empty():
        
        
        current_cost, current_node = q.get()   

        if (current_node == goal):
            break      
        else:
            closedSet.add(current_node)
            for each_child in G.neighbors(current_node):  #CHANGE
                                
                if each_child in closedSet:   
                    # if current node or existing parent of visited child, who is better parent?
                    past_cost = calculate_totalGD(each_child)         # this is gScore      #CHANGE  
                    t_gScore =  calculate_GD(current_node,each_child) + calculate_totalGD(current_node) #CHANGE
                    #print('T score: ',t_gScore)
                    if (t_gScore < past_cost):
                        cameFrom[each_child] = current_node
                        
                if not each_child in closedSet:   #to avoid 'C' need not revisit
                    cameFrom[each_child] = current_node
                    past_cost = calculate_totalGD(each_child)  #CHANGE
                    heuristic_cost = calculate_GD(each_child, goal)  #CHANGE
                    #print('past cost: {} heuristic cost: {}'.format(past_cost,heuristic_cost))
                    total_cost = past_cost + heuristic_cost
                    q.put( (total_cost, each_child) )
                    closedSet.add(each_child)   
                                        
        count += 1  
        #print('Iter: {} Node: {} Q:{}'.format(count,current_node,list(q.queue)))
                    
    best_path = calculate_path(current_node)
    return (current_cost, best_path)


(cost, path) = run_AStar(start,goal)
print(cost, path)
print(cameFrom)
show_map(G,start=start,goal=goal,path=path,SLDTarget=goal)

39 [5, 16, 37, 12, 34]
{5: None, 32: 5, 16: 5, 14: 5, 37: 16, 30: 16, 12: 37, 22: 37, 29: 37, 34: 12, 31: 12, 28: 12, 17: 12}


Cool. This is same as Udacity's answer as well :)   

Let us next try to modify to be compliant with Udacity's new project structure to be submitted. <font color='blue'>At this moment, user is requested to have Udacity Route planner project template ready, especially main.py opened</font>

**Step 1**: Let us modify the names so as to be compliant with run_search() in main.py, which executes the algorithm and check if it still runs fine. We will also define minimal PathPlanner class to assist in this step. **Do compare this step with our raw algorithm side by side to understand the changes**

In [6]:
from queue import PriorityQueue
import geopy.distance


class PathPlanner():
    
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        
        self.cameFrom = {}           # this should be defined before run_search()
        self.cameFrom[start] = None  
        
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None                   
        
    def reconstruct_path(self, current):  #calculate path modified
        total_path = [current]
        while current is not None:    
            current = self.cameFrom.get(current)
            total_path.append(current)
        return [x for x in reversed(total_path[:-1])]
    
    def calculate_GD(self, start, end): 
        (x0,y0) = self.map.node[start]['pos']     
        (x1,y1) = self.map.node[end]['pos']      
        return floor(geopy.distance.geodesic((y0,x0), (y1,x1)).miles)     
    
    def calculate_totalGD(self, node):
        cost = 0
        if self.cameFrom.get(node) is None: return 0
        while True:   #when current_node is not None..        
            cost += calculate_GD(self.cameFrom.get(node),node)
            node = self.cameFrom.get(node)
            if self.cameFrom.get(node) is None: break
        return cost    
    
    def run_search(self):
        
        # Initialization
        q = PriorityQueue()
        q.put((0,self.start))
        closedSet = set()  # set() would be better than a list?


        # Main loop
        while not q.empty():

            cost, current = q.get()   

            if (current == goal):
                self.path = self.reconstruct_path(current)
                return self.path
            
            else:
                closedSet.add(current)
                for neighbor in self.map.neighbors(current):  #CHANGE

                    if neighbor in closedSet:                           
                        past_cost = calculate_totalGD(neighbor)                 
                        t_gScore =  calculate_GD(current,neighbor) + calculate_totalGD(current) 
                        if (t_gScore < past_cost):
                            self.cameFrom[neighbor] = current

                    if not neighbor in closedSet:   
                        
                        self.cameFrom[neighbor] = current
                        past_cost = calculate_totalGD(neighbor)  
                        heuristic_cost = calculate_GD(neighbor, goal)  
                        total_cost = past_cost + heuristic_cost
                        
                        q.put( (total_cost, neighbor) )
                        closedSet.add(neighbor)   
                        
        print("No Path Found")
        self.path = None
        return False       
    
planner = PathPlanner(map_40, 5, 34)
print(planner.path)

[5, 16, 37, 12, 34]


Success!  

**Step 2**: Let us modify the rest as much as in main run_search() function..

In [7]:
from queue import PriorityQueue
import geopy.distance


class PathPlanner():
    
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        
        self.cameFrom = {}           # this should be defined before run_search()
        self.cameFrom[start] = None  
        
        self.openSet = PriorityQueue()        
        self.closedSet = set()
        
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None                   
        
    def reconstruct_path(self, current):  #calculate path modified
        total_path = [current]
        while current is not None:    
            current = self.cameFrom.get(current)
            total_path.append(current)
        return total_path[:-1]
    
    def calculate_GD(self, start, end): 
        (x0,y0) = self.map.node[start]['pos']     
        (x1,y1) = self.map.node[end]['pos']      
        return floor(geopy.distance.geodesic((y0,x0), (y1,x1)).miles)     
      
    def calculate_totalGD(self, node):
        cost = 0
        if self.cameFrom.get(node) is None: return 0
        while True:   #when current_node is not None..        
            cost += self.calculate_GD(self.cameFrom.get(node),node)
            node = self.cameFrom.get(node)
            if self.cameFrom.get(node) is None: break
        return cost    
    
    def is_open_empty(self):
        return self.openSet.empty()    
    
    def get_current_node(self):
        return self.openSet.get()
    
    def get_neighbors(self,current):
        return self.map.neighbors(current)
    
    def add_to_openSet(self, neighbor):     
        """
        Adding to priority Q, both a neighbor n and total cost f(n) involved in arriving at that neighbor
        f(n) = g(n) + h(n)
        total cost f(n) is stored as priority        
        """
        gScore = self.get_gScore(neighbor)  
        hScore = self.get_hScore(neighbor, self.goal)  
        fScore = gScore + hScore
        self.openSet.put( (fScore, neighbor) )
        #print(fScore,neighbor)
        
    def get_gScore(self, neighbor):
        """
        Getting the actual cost from start to neighbor by totalling step costs g(n)
        """
        return self.calculate_totalGD(neighbor)
    
    def get_tenative_gScore(self, current, neighbor):
        return self.calculate_GD(current,neighbor) + self.calculate_totalGD(current) 
    
    def get_hScore(self, neighbor, goal):
        """
        Getting the heuristic cost estimate from neighbor to goal h(n)
        """
        return self.calculate_GD(neighbor, goal) 
    
    def record_best_path_to(self, current, neighbor):
        """
        Updating a neighbor with 'better' parent. The one that has least cost path to that kid.
        """
        self.cameFrom[neighbor] = current
    
    def run_search(self):
        
        # Initialization
        self.add_to_openSet(self.start)

        # Main loop
        while not self.is_open_empty():

            cost, current = self.get_current_node()

            if (current == self.goal):
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            
            else:
                #self.openSet.remove(current) not necessary as its a Queue
                self.closedSet.add(current)
                
                for neighbor in self.get_neighbors(current):  

                    if neighbor in self.closedSet:                           
                                               
                        if (self.get_tenative_gScore(current, neighbor) < self.get_gScore(neighbor)):                            
                            self.record_best_path_to(current, neighbor)

                    else:   
                        
                        self.record_best_path_to(current, neighbor)
                                                                    
                        self.add_to_openSet(neighbor) 
                        
                        self.closedSet.add(neighbor)   
                        
        print("No Path Found")
        self.path = None
        return False       


map_40 = load_map_graph(map_40_dict)
planner = PathPlanner(map_40, 5, 34)
print(planner.path)

#graph = load_map_graph(romania_location_map)
#planner = PathPlanner(graph, 'A', 'B')
#print(planner.path)

[5, 16, 37, 12, 34]


**Step 3**: This is small one but needs mention. Note the below snippet in the loop

```python
                    if neighbor in self.closedSet:                           
                                               
                        if (self.get_tenative_gScore(current, neighbor) < self.get_gScore(neighbor)):                            
                            self.record_best_path_to(current, neighbor)

                    else:   
                        
                        self.record_best_path_to(current, neighbor)
                                                                    
                        self.add_to_openSet(neighbor) 
                        
                        self.closedSet.add(neighbor)  
```

record_best_path_to is done both in if and else part. We could optimize it.

We could rewrite as below dividing the else part deliberately to facilitate altering logic. Note now instead of writing if ```t_gScore < gScore```, we will always write and skip if ```t_gScore >= gScore```

```python
                    if neighbor in self.closedSet:                                                                          
                        if (self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor)):                            
                            continue #to next iteration in for loop
                            
                        self.record_best_path_to(current, neighbor)
                        
                    if not neighbor in self.closedSet:   
                                           
                        self.record_best_path_to(current, neighbor)
                        
                    if not neighbor in self.closedSet:
                                                                    
                        self.add_to_openSet(neighbor) 
                        
                        self.closedSet.add(neighbor)  
```

This could be further optimized as below. 

```python
                    if neighbor in self.closedSet:   
    
                        if (self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor)):                 
                            continue #to next iteration in for loop 
        
                    self.record_best_path_to(current, neighbor)                             
                        
                    if not neighbor in self.closedSet:   
            
                        self.add_to_openSet(neighbor)   
            
                        self.closedSet.add(neighbor)  
```

In [8]:
from queue import PriorityQueue
import geopy.distance


class PathPlanner():
    
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        
        self.cameFrom = {}           # this should be defined before run_search()
        self.cameFrom[start] = None  
        
        self.openSet = PriorityQueue()        
        self.closedSet = set()
        
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None                   
        
    def reconstruct_path(self, current):  #calculate path modified
        total_path = [current]
        while current is not None:    
            current = self.cameFrom.get(current)
            total_path.append(current)
        return total_path[:-1]
    
    def calculate_GD(self, start, end): 
        (x0,y0) = self.map.node[start]['pos']     
        (x1,y1) = self.map.node[end]['pos']      
        return floor(geopy.distance.geodesic((y0,x0), (y1,x1)).miles)     
      
    def calculate_totalGD(self, node):
        cost = 0
        if self.cameFrom.get(node) is None: return 0
        while True:   #when current_node is not None..        
            cost += self.calculate_GD(self.cameFrom.get(node),node)
            node = self.cameFrom.get(node)
            if self.cameFrom.get(node) is None: break
        return cost    
    
    def is_open_empty(self):
        return self.openSet.empty()    
    
    def get_current_node(self):
        return self.openSet.get()
    
    def get_neighbors(self,current):
        return self.map.neighbors(current)
    
    def add_to_openSet(self, neighbor):     
        """
        Adding to priority Q, both a neighbor n and total cost f(n) involved in arriving at that neighbor
        f(n) = g(n) + h(n)
        total cost f(n) is stored as priority        
        """
        gScore = self.get_gScore(neighbor)  
        hScore = self.get_hScore(neighbor, self.goal)  
        fScore = gScore + hScore
        self.openSet.put( (fScore, neighbor) )
        #print(fScore,neighbor)
        
    def get_gScore(self, neighbor):
        """
        Getting the actual cost from start to neighbor by totalling step costs g(n)
        """
        return self.calculate_totalGD(neighbor)
    
    def get_tenative_gScore(self, current, neighbor):
        return self.calculate_GD(current,neighbor) + self.calculate_totalGD(current) 
    
    def get_hScore(self, neighbor, goal):
        """
        Getting the heuristic cost estimate from neighbor to goal h(n)
        """
        return self.calculate_GD(neighbor, goal) 
    
    def record_best_path_to(self, current, neighbor):
        """
        Updating a neighbor with 'better' parent. The one that has least cost path to that kid.
        """
        self.cameFrom[neighbor] = current
    
    def run_search(self):
        
        # Initialization
        self.add_to_openSet(self.start)

        # Main loop
        while not self.is_open_empty():

            cost, current = self.get_current_node()

            if (current == self.goal):
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            
            else:
                #self.openSet.remove(current) not necessary as its a Queue
                self.closedSet.add(current)
                
                for neighbor in self.get_neighbors(current):                             
                
                    if neighbor in self.closedSet:                                                                          
                        if (self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor)):                            
                            continue #to next iteration in for loop                                
                    self.record_best_path_to(current, neighbor)                     
                
                    if not neighbor in self.closedSet:                                                                    
                        self.add_to_openSet(neighbor)                         
                        self.closedSet.add(neighbor)                                   
                                            
                        
        print("No Path Found")
        self.path = None
        return False       


map_40 = load_map_graph(map_40_dict)
planner = PathPlanner(map_40, 5, 34)
print(planner.path)

graph = load_map_graph(romania_location_map)
planner = PathPlanner(graph, 'A', 'B')
print(planner.path)

[5, 16, 37, 12, 34]
['A', 'S', 'RV', 'P', 'B']


Let us now move on to cosmetics.  
1. You see ```calculate_totalGD(n)``` is g(n), so we could rename as ```create_gScore(n)```
2. ```calculate_GD(start,end)``` is distance between start and end nodes, so can be renamed as ```distance(start,end)```
3. ```heuristic_cost_estimate``` needed in project is also a distance function, only that, end node is goal. So accordingly could be included (to internally call distance function again)
4. f(n) = g(n) + h(n). So ```calculate_fscore(n)``` could be introduced to calculate ```fScore``` (which we currently do in add_to_openSet)  

Now our code becomes much more similar to Udacity project's. 

In [9]:
from queue import PriorityQueue
import geopy.distance


class PathPlanner():
    
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        
        self.cameFrom = {}           # this should be defined before run_search()
        self.cameFrom[start] = None  
        
        self.openSet = PriorityQueue()        
        self.closedSet = set()
        
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None                   
        
    def reconstruct_path(self, current):  #calculate path modified
        total_path = [current]
        while current is not None:    
            current = self.cameFrom.get(current)
            total_path.append(current)
        return total_path[:-1]

    def create_gScore(self, node):
        cost = 0
        if self.cameFrom.get(node) is None: return 0
        while True:   #when current_node is not None..        
            cost += self.distance(self.cameFrom.get(node),node)
            node = self.cameFrom.get(node)
            if self.cameFrom.get(node) is None: break
        return cost    
      
    def get_current_node(self):
        """ Returns the node in the open set with the lowest value of f(node)."""
        return self.openSet.get()
    
    def get_neighbors(self,current):
        """Returns the neighbors of a node"""
        return self.map.neighbors(current)
    
    def get_gScore(self, neighbor):
        """Returns the g Score of a node"""
        return self.create_gScore(neighbor)    
    
    def get_tenative_gScore(self, current, neighbor):
        """Returns the tenative g Score of a node"""
        return self.distance(current,neighbor) + self.create_gScore(current)   
    
    def is_open_empty(self):
        """returns True if the open set is empty. False otherwise. """        
        return self.openSet.empty() 
    
    def distance(self, start, end): 
        """ Computes the geodesic distance which is better than Euclidean"""
        (x0,y0) = self.map.node[start]['pos']     
        (x1,y1) = self.map.node[end]['pos']      
        return floor(geopy.distance.geodesic((y0,x0), (y1,x1)).miles)        
   
    def heuristic_cost_estimate(self, neighbor, goal):
        """ Returns the heuristic cost estimate of a node """
        return self.distance(neighbor, goal)     
    
    def calculate_fscore(self, neighbor):
        """Calculate the f score of a node. """
        # REMEMBER F = G + H        
        gScore = self.get_gScore(neighbor)  
        hScore = self.heuristic_cost_estimate(neighbor, self.goal)  
        return gScore + hScore      
    
    def record_best_path_to(self, current, neighbor):
        """
        Updating a neighbor with 'better' parent. The one that has least cost path to that kid.
        """
        self.cameFrom[neighbor] = current    
        
    def add_to_openSet(self, neighbor):     
        """
        Adding to priority Q, both a neighbor n and total cost f(n) involved in arriving at that neighbor
        total cost f(n) is stored as priority        
        """
        fScore = self.calculate_fscore(neighbor)
        self.openSet.put( (fScore, neighbor) )
        #print(fScore,neighbor)               
    
    def run_search(self):
        
        # Initialization
        self.add_to_openSet(self.start)

        # Main loop
        while not self.is_open_empty():

            cost, current = self.get_current_node()

            if (current == self.goal):
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            
            else:
                #self.openSet.remove(current) not necessary as its a Queue
                self.closedSet.add(current)
                
                for neighbor in self.get_neighbors(current):                             
                
                    if neighbor in self.closedSet:                                                                          
                        if (self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor)):                            
                            continue #to next iteration in for loop                                
                    self.record_best_path_to(current, neighbor)                     
                
                    if not neighbor in self.closedSet:                                                                    
                        self.add_to_openSet(neighbor)                         
                        self.closedSet.add(neighbor)                                   
                                            
                        
        print("No Path Found")
        self.path = None
        return False       


map_40 = load_map_graph(map_40_dict)
planner = PathPlanner(map_40, 5, 34)
print(planner.path)

graph = load_map_graph(romania_location_map)
planner = PathPlanner(graph, 'A', 'B')
print(planner.path)

[5, 16, 37, 12, 34]
['A', 'S', 'RV', 'P', 'B']


Adding few more cosmetics  (mostly the init part stuff of the class moved to these functions)
1. create_closedSet
2. create_openSet
3. create_cameFrom

In [18]:
from queue import PriorityQueue
import geopy.distance


class PathPlanner():
    
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal        
        self.cameFrom = self.create_cameFrom()        
        self.openSet = self.create_openSet()        
        self.closedSet = self.create_closedSet()  
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None  
        
    def reconstruct_path(self, current):  #calculate path modified
        total_path = [current]
        while current is not None:    
            current = self.cameFrom.get(current)
            total_path.append(current)
        return total_path[:-1]        
        
    def create_closedSet(self):
        """ Creates and returns a data structure suitable to hold the set of nodes already evaluated"""
        return set()        
    
    def create_openSet(self):
        """ Creates and returns a data structure suitable to hold the set of currently discovered nodes 
        that are not evaluated yet. Initially, only the start node is known."""
        if self.start != None:
            openSet = PriorityQueue()           
            openSet.put((0,self.start))               
            return openSet
        raise ValueError("Must create start node before creating an open set. Try running PathPlanner.set_start(start_node)")    
        
    def create_cameFrom(self):
        """Creates and returns a data structure that shows which node can most efficiently be reached from another,
        for each node."""
        # TODO: return a data structure that shows which node can most efficiently be reached from another,
        # for each node.     
        cameFrom = {}
        if self.start != None:
            cameFrom[start] = None
            return cameFrom
        raise ValueError("Must create start node before creating a cameFrom list. Try running PathPlanner.set_start(start_node)")            
        
    def create_gScore(self, node):
        cost = 0
        if self.cameFrom.get(node) is None: return 0
        while True:   #when current_node is not None..        
            cost += self.distance(self.cameFrom.get(node),node)
            node = self.cameFrom.get(node)
            if self.cameFrom.get(node) is None: break
        return cost    

    def _reset(self):
        """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
        self.closedSet = None
        self.openSet = None
        self.cameFrom = None
        self.gScore = None
        self.fScore = None
        self.path = self.run_search() if self.map and self.start and self.goal else None

    def set_map(self, M):
        """Method used to set map attribute """
        self._reset(self)
        self.start = None
        self.goal = None
        self.map = M 

    def set_start(self, start):
        """Method used to set start attribute """
        self._reset(self)
        self.start= start   
    
    def get_current_node(self):
        """ Returns the node in the open set with the lowest value of f(node)."""
        return self.openSet.get()
    
    def get_neighbors(self,current):
        """Returns the neighbors of a node"""
        return self.map.neighbors(current)
    
    def get_gScore(self, neighbor):
        """Returns the g Score of a node"""
        return self.create_gScore(neighbor)    
    
    def get_tenative_gScore(self, current, neighbor):
        """Returns the tenative g Score of a node"""
        return self.distance(current,neighbor) + self.create_gScore(current)   
    
    def is_open_empty(self):
        """returns True if the open set is empty. False otherwise. """               
        return self.openSet.empty() 
    
    def distance(self, start, end): 
        """ Computes the geodesic distance which is better than Euclidean"""
        (x0,y0) = self.map.node[start]['pos']     
        (x1,y1) = self.map.node[end]['pos']      
        return floor(geopy.distance.geodesic((y0,x0), (y1,x1)).miles)        
   
    def heuristic_cost_estimate(self, neighbor, goal):
        """ Returns the heuristic cost estimate of a node """
        return self.distance(neighbor, goal)     
    
    def create_fScore(self, neighbor):
        """Calculate the f score of a node. """
        # REMEMBER F = G + H        
        gScore = self.get_gScore(neighbor)  
        hScore = self.heuristic_cost_estimate(neighbor, self.goal)  
        return gScore + hScore      
    
    def record_best_path_to(self, current, neighbor):
        """
        Updating a neighbor with 'better' parent. The one that has least cost path to that kid.
        """
        self.cameFrom[neighbor] = current    
        
    def add_to_openSet(self, neighbor):     
        """
        Adding to priority Q, both a neighbor n and total cost f(n) involved in arriving at that neighbor
        total cost f(n) is stored as priority        
        """
        fScore = self.create_fScore(neighbor)
        self.openSet.put( (fScore, neighbor) )
        #print(fScore,neighbor)               
    
    def run_search(self):        
        
        if self.map == None:
            raise ValueError("Must create map before running search. Try running PathPlanner.set_map(start_node)")
        if self.goal == None:
            raise ValueError("Must create goal node before running search. Try running PathPlanner.set_goal(start_node)")
        if self.start == None:
            raise ValueError("Must create start node before running search. Try running PathPlanner.set_start(start_node)")        

        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet()
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()           
            
        # Main loop
        while not self.is_open_empty():

            cost, current = self.get_current_node()

            if (current == self.goal):
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            
            else:
                #self.openSet.remove(current) not necessary as its a Queue
                self.closedSet.add(current)
                
                for neighbor in self.get_neighbors(current):                             
                
                    if neighbor in self.closedSet:                                                                          
                        if (self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor)):                            
                            continue #to next iteration in for loop                                
                    self.record_best_path_to(current, neighbor)                     
                
                    if not neighbor in self.closedSet:                                                                    
                        self.add_to_openSet(neighbor)                         
                        self.closedSet.add(neighbor)                                   
                                            
                        
        print("No Path Found")
        self.path = None
        return False       


map_40 = load_map_graph(map_40_dict)
planner = PathPlanner(map_40, 5, 34)
print(planner.path)
planner = PathPlanner(map_40, 8, 24)
print(planner.path)
planner = PathPlanner(map_40, 5, 5)
print(planner.path)

graph = load_map_graph(romania_location_map)
planner = PathPlanner(graph, 'A', 'B')
print(planner.path)

[5, 16, 37, 12, 34]
[8, 14, 16, 37, 12, 17, 10, 24]
[5]
['A', 'S', 'RV', 'P', 'B']


In [19]:
from pathplanner import PathPlanner as PathPlanner1
start = 5
goal = 34
map_40 = load_map_graph(map_40_dict)
planner = PathPlanner1(map_40, start, goal)
print(planner.path)

[5, 16, 37, 12, 34]
